In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv
/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv


In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
train_df = pd.read_csv('/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv')
train_df.sample(5)

,clean_text,category
16932,once modi unpalatable cong then why not accoun...,0.0
9332,tourism receives major boost under modi govern...,1.0
149907,sir modi supporter you may believemodi has str...,1.0
28196,modi much india hitler was germany whore you w...,1.0
28704,giriraj singhs hesitancy contest against kanha...,-1.0


In [24]:
test_df = pd.read_csv('/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv')
test_df.sample(5)

,clean_comment,category
4835,most educated indians are have inherent caste ...,1
1310,does anyone have any updates potential fire su...,1
35693,hume debris dikhao video evidence kahan hai ye...,-1
31140,all this carnage could have been prevented had...,-1
33174,democracy danger even when congress rules only...,-1


# Data Inspection and Data Cleaning

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [26]:
train_df.isnull().sum()

clean_text    4
category      7
dtype: int64

In [27]:
train_df = train_df.dropna()
train_df.category = train_df.category.astype(int)
train_df = train_df.reset_index(drop = True)

In [28]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [29]:
test_df.isnull().sum()

clean_comment    100
category           0
dtype: int64

In [30]:
test_df = test_df.dropna()
test_df = test_df.reset_index(drop = True)

In [31]:
train_df.sample()

,clean_text,category
34264,what justification bjp journalist govt also pl...,0


In [32]:
import re
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()

def text_preprocessing(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub("[^a-zA-Z]", " ", text)    # keep only alphabets
    text = text.lower()                      # Lowering all words in each review rows for better text preprocessing
    tokens = text.split()                    # word tokenization from each reviews
    cleaned_tokens = [lemma.lemmatize(word) for word in tokens] 

    return cleaned_tokens
 

train_df["tokens"] = train_df.clean_text.apply(text_preprocessing)
test_df["tokens"] = test_df.clean_comment.apply(text_preprocessing)

In [33]:
test_df.sample(4)

,clean_comment,category,tokens
35197,lot people taking shit without any knowledge s...,-1,"[lot, people, taking, shit, without, any, know..."
18990,satish acharya was doing what most journalists...,1,"[satish, acharya, wa, doing, what, most, journ..."
19196,sikhs canada are renegades who fled india 1980...,1,"[sikh, canada, are, renegade, who, fled, india..."
31169,this man and his circlejerk the gangrene our c...,0,"[this, man, and, his, circlejerk, the, gangren..."


In [36]:
test_df.iloc[18990, 2]

['satish',
 'acharya',
 'wa',
 'doing',
 'what',
 'most',
 'journalist',
 'failed',
 'take',
 'the',
 'bullshit',
 'the',
 'system',
 'saw',
 'ha',
 'bashed',
 'the',
 'congress',
 'party',
 'and',
 'any',
 'other',
 'political',
 'party',
 'saw',
 'fit',
 'through',
 'his',
 'humorous',
 'cartoon',
 'and',
 'none',
 'his',
 'cartoon',
 'are',
 'poor',
 'taste',
 'bet',
 'the',
 'power',
 'that',
 'are',
 'upset',
 'with',
 'him',
 'because',
 'doe',
 'with',
 'humor',
 'and',
 'people',
 'power',
 'cannot',
 'afford',
 'made',
 'fun',
 'that',
 'where',
 'they',
 'are',
 'mistaken',
 'not',
 'making',
 'fun',
 'individual',
 'usually',
 'show',
 'the',
 'hypocrisy',
 'policy',
 'and',
 'governance',
 'this',
 'the',
 'sort',
 'person',
 'should',
 'support',
 'wholeheartedly',
 'irrespective',
 'political',
 'affiliation']

# Converting Word to Vectors

In [47]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.text import one_hot

# Number of unique words in the entire clean_comment
words = [word for sentences in train_df.tokens for word in sentences]
vocabulary = set(words)
vocab_size = len(vocabulary)

# getting word index for every word
def get_one_hot_indices(token_list):
    sentences = " ".join(token_list)  # gathering all tokens rows into a single variable
    return one_hot(sentences, vocab_size)

# word encoding
train_df['encoded_tokens'] = train_df['tokens'].apply(get_one_hot_indices)
test_df['encoded_tokens'] = test_df['tokens'].apply(get_one_hot_indices)

2025-09-16 10:26:10.044665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758018370.214657      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758018370.267782      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [49]:
train_df.sample()

,clean_text,category,tokens,encoded_tokens
70671,think balakot happened because the govt was pr...,1,"[think, balakot, happened, because, the, govt,...","[18543, 53072, 44360, 56155, 80942, 19910, 417..."


In [50]:
test_df.sample()

,clean_comment,category,tokens,encoded_tokens
30039,sounds like you entered the eight levels jhana,0,"[sound, like, you, entered, the, eight, level,...","[61239, 858, 22195, 75121, 80942, 27406, 53561..."


In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Determining the max sentence size
train_df['token_length'] = train_df['tokens'].apply(len)
max_len = train_df.token_length.max()     # max_len = 52

# Using Padding to sentence lenghts same for every sentences
embedded_train_sequences = pad_sequences(train_df['encoded_tokens'].to_list(), maxlen = max_len, padding = "post" )
embedded_test_sequences = pad_sequences(test_df['encoded_tokens'].to_list(), maxlen = max_len, padding = "post")

In [61]:
train_df['encoded_tokens'].to_list()

[[75902,
  39029,
  42657,
  50667,
  3117,
  44906,
  39347,
  44992,
  34772,
  20862,
  80942,
  38069,
  16258,
  60777,
  80942,
  32074,
  71837,
  30214,
  58781,
  60272,
  89200,
  66443,
  32074,
  44764,
  44105,
  32715,
  13539,
  44105,
  44764,
  12879,
  46608,
  44105,
  16248],
 [11800,
  67155,
  80942,
  19247,
  44105,
  50235,
  67155,
  80942,
  21517,
  65517,
  80768,
  55887,
  39029],
 [71493,
  63365,
  37188,
  66209,
  80768,
  55887,
  39029,
  95225,
  39347,
  26355,
  22195,
  79378,
  80942,
  90313,
  94657,
  55887,
  39029,
  18543,
  39029,
  44764,
  37188,
  85186],
 [71336,
  498,
  68564,
  91584,
  32235,
  11068,
  6778,
  39029,
  63365,
  77395,
  89278,
  80548,
  43784,
  77354,
  71493,
  44242,
  71493,
  32715,
  80548,
  9727,
  7743,
  71493,
  65517,
  28917,
  91082,
  72811,
  84672,
  46024,
  86544,
  45103,
  43155,
  37786,
  64436,
  81676],
 [15611,
  48716,
  76130,
  55879,
  80942,
  86544,
  17598,
  68517,
  34355,
  4

In [59]:
embedded_train_sequences

array([[75902, 39029, 42657, ...,     0,     0,     0],
       [11800, 67155, 80942, ...,     0,     0,     0],
       [71493, 63365, 37188, ...,     0,     0,     0],
       ...,
       [63365, 22195, 45507, ...,     0,     0,     0],
       [43784, 62125, 79603, ...,     0,     0,     0],
       [90572, 22195, 29259, ...,     0,     0,     0]], dtype=int32)

In [63]:
embedded_train_sequences[1]

array([11800, 67155, 80942, 19247, 44105, 50235, 67155, 80942, 21517,
       65517, 80768, 55887, 39029,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

# Train-Test Split

In [64]:
train_df.columns

Index(['clean_text', 'category', 'tokens', 'encoded_tokens', 'token_lenght',
       'token_length'],
      dtype='object')

In [65]:
from sklearn.model_selection import train_test_split

X = embedded_train_sequences
y = train_df.category

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)


In [68]:
X_train.shape, X_test.shape

((130375, 52), (32594, 52))

In [69]:
y_train

147638    1
72162     1
145148    0
84292     0
15416     0
         ..
70209     1
91261    -1
5014      0
153656    1
86364    -1
Name: category, Length: 130375, dtype: int64

In [80]:
label_mapping = {-1: 0, 0: 1, 1: 2}
y_train = y_train.map(label_mapping)
y_test = y_test.map(label_mapping)


# Building the Simple RNN model

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

embedding_dim = 100
vocab_size = vocab_size #number of unique vocabulary
max_len = max_len #maximum sentence length


model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = max_len)) #Layer1
model.add(SimpleRNN(128, activation = 'elu'))
model.add(Dense(3, activation = 'softmax'))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [82]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights = True)

model.fit(X_train, y_train,
         validation_data = (X_test, y_test),
         epochs = 100, 
         batch_size = 128,
         callbacks = early_stopping_callback)

Epoch 1/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.5281 - loss: 0.9905 - val_accuracy: 0.4511 - val_loss: 1.0546
Epoch 2/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.4502 - loss: 1.0561 - val_accuracy: 0.4444 - val_loss: 1.0708
Epoch 3/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.4599 - loss: 1.0515 - val_accuracy: 0.4537 - val_loss: 1.0563
Epoch 4/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5079 - loss: 0.9779 - val_accuracy: 0.8196 - val_loss: 0.4957
Epoch 5/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8445 - loss: 0.4359 - val_accuracy: 0.8870 - val_loss: 0.3465
Epoch 6/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9138 - loss: 0.2650 - val_accuracy: 0.9132 - val_loss: 0.2814
Epoch 7/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9445 - loss: 0.1842 - val_accuracy: 0.9157 - val_loss: 0.2759
Epoch 8/100
1019/1019 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9594 - loss: 

In [83]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 52, 100)        │     9,572,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ (None, 128)            │        29,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,807,799 (109.89 MB)

 Trainable params: 9,602,599 (36.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,205,200 (73.26 MB)

# Predictions on Reddit.csv

In [85]:
predictions= model.predict(embedded_test_sequences)
predictions

1161/1161 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


array([[8.4706143e-02, 8.7219914e-03, 9.0657187e-01],
       [3.3677134e-01, 1.7333750e-02, 6.4589494e-01],
       [9.7523892e-01, 1.7744295e-02, 7.0168097e-03],
       ...,
       [2.7024637e-03, 9.9430084e-01, 2.9966966e-03],
       [1.4348173e-03, 9.0856157e-04, 9.9765658e-01],
       [8.8106859e-03, 9.7044170e-01, 2.0747503e-02]], dtype=float32)

In [90]:
from sklearn.metrics import accuracy_score, classification_report

# Get the predicted class by finding the index with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

y_true_reddit = test_df.category.replace({-1:0, 0:1, 1:2})

accuracy = accuracy_score(y_true_reddit, predicted_classes)
print(f"Accuracy on Reddit data: {accuracy:.4f}")

print("\nClassification Report on Reddit data:")
print(classification_report(y_true_reddit, predicted_classes))

Accuracy on Reddit data: 0.8710

Classification Report on Reddit data:
              precision    recall  f1-score   support

           0       0.77      0.74      0.75      8277
           1       0.93      0.91      0.92     13042
           2       0.88      0.90      0.89     15830

    accuracy                           0.87     37149
   macro avg       0.86      0.85      0.86     37149
weighted avg       0.87      0.87      0.87     37149

